In [3]:
# Part a)
# Travelling salesman - Successive insertion
distance = [[100, 3, 5, 2, 6, 4], [2, 100, 7, 5, 1, 3],[2, 4, 100, 3, 2, 6], [5, 3, 6, 100, 4, 4],[1, 4, 5, 2, 100, 6],[6, 5, 7, 3, 3, 100]]
nodes = 6 # index as 0 ... nodes-1

tour = [0, 0] #start and end at node 0
length = 0 #total length of current tour

import numpy as np

for i in range(1, nodes) : #add nodes in order of 1 -> nodes-1
    length_change = np.zeros(len(tour) - 1) #list of change in length for each possible position to insert new node
    
    for position in range(len(tour) - 1): #iterate through possible positions in current tour
        #temp_tour = tour[: position+1] + [i] + tour[position+1 :]
        
        #calculate the change in length between current tour and temp tour
        pre_ins_node = tour[position]
        nex_ins_node = tour[position + 1]
        if pre_ins_node == nex_ins_node:
            length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node]
        else:
            length_change[position] = distance[pre_ins_node][i] + distance[i][nex_ins_node] - distance[pre_ins_node][nex_ins_node]
    
    min_position = np.argmin(length_change) #get the position which create the shortest length change
    length += length_change[min_position]
    tour = tour[: min_position+1] + [i] + tour[min_position+1 :]

print("Tour:", tour)
print("Total length of the tour is:", length)

Tour: [0, 2, 3, 1, 5, 4, 0]
Total length of the tour is: 18.0


In [2]:
# Part b)
# Visualize the route determined

from graphviz import Digraph

f = Digraph(comment='TPP')

for i in range(len(tour)): 
    f.node(str(tour[i]))

for i in range(len(tour)-1):
    this_node = tour[i]
    next_node = tour[i+1]
    f.edge(str(this_node), str(next_node), label=str(distance[this_node][next_node]))
    
f.view()


'Digraph.gv.pdf'

In [3]:
# Part c)
# Improve solution by using 2-opt

# define function to swap 2 edges of nodes with index m and n (starting index 0)  
def two_opt_swap (tour, m, n):
    import copy
    new_tour = copy.deepcopy(tour)
    new_tour[m:n+1] = reversed(new_tour[m:n+1])
    return new_tour

# define function to calculate the length of the tour 
##### comment: only calculate the cost in swap edges and reverse tour => reduce computation time
def totalcost(tour):
    length = 0
    for i in range(len(tour)-1):
        length += distance[tour[i]][tour[i+1]]
    return length

In [4]:
# define 2-opt function
def two_opt (tour):
    import copy
    
    best_tour = tour
    improvement = True
    
    while improvement: 
        improvement = False
        for i in range(1,len(tour)-2):
            for j in range(i+1, len(tour)-1):
                temp_tour = two_opt_swap(tour, i, j)
                if totalcost(temp_tour) < totalcost(best_tour):
                    best_tour = copy.deepcopy(temp_tour)
                    improvement = True
    
    return best_tour, totalcost(best_tour)


In [5]:
# apply 2-opt method to Nearest neighbor solution

# parameters
distance = [[100, 3, 5, 2, 6, 4], [2, 100, 7, 5, 1, 3],[2, 4, 100, 3, 2, 6], [5, 3, 6, 100, 4, 4],[1, 4, 5, 2, 100, 6],[6, 5, 7, 3, 3, 100]]
position = 0
tour = [0]
length = 0

# get result from Nearest neighbor approach (as from lecture)
for i in range(5):
    nn = 0
    nd = 100 #starting distance => maximum value
    for j in range(6):  
        if (j not in tour) and (distance[position][j]<nd): #node not in the tour yet
            nd = distance[position][j]
            nn = j
    tour.append(nn) # append the list
    length = length + nd
    position = nn
tour.append(0)
length = length + distance[position][0]
print("Nearest neighbor solution is", tour)
print("Total length of Nearest neighbor solution:", length)

#improve by 2-opt
new_tour, new_length = two_opt(tour)
print("Solution after improving by 2-opt is", new_tour)
print("Total length of improved solution:", new_length)

Nearest neighbor solution is [0, 3, 1, 4, 2, 5, 0]
Total length of Nearest neighbor solution: 23
Solution after improving by 2-opt is [0, 3, 1, 5, 2, 4, 0]
Total length of improved solution: 18
